In [1]:
from ipynb.fs.full.File_Variables import *

In [2]:
def get_the_EB_runtimes_df(RTVER_NO):
    import math
    import pandas as pd
    from datetime import datetime, timedelta, date
    
    # data_EB = pd.read_csv("C:/Users/MBhutada.int/OneDrive - Chicago Transit Authority/Desktop/CTA_Mithilesh/CTA Assignments/Metra GTFS for Cubs/Excel and CSV files/Q-04 80 runtimes WK SA SU.csv")
    data_EB = pd.read_csv(csv_name_runtime)

    data_EB = data_EB[['RTVER_NO', 'START_PLACE_ID', 'END_PLACE_ID', 'PERIOD_START_TIME', 'PERIOD_END_TIME', 'RVARIANT_ID',  'RUN_TIME']]
    data_EB = data_EB[(data_EB['RVARIANT_ID'] == 'EB') & (data_EB['RTVER_NO'] == RTVER_NO)]

    data_EB['REVISED_PST'] = data_EB['PERIOD_START_TIME'].apply(lambda x: x/36000)
    data_EB['REVISED_PET'] = data_EB['PERIOD_END_TIME'].apply(lambda x: x/36000)

    def convert_time(element):
        subelement = (element - int(str(element)[:2])) * 0.6
        element = float(str(element)[:2]) - 12
        element = element + subelement
        element = "{:.2f}".format(element)
        element = f"{float(element):05.2f}"
        element = str(element)
        if element[3:] == '60':
            element = str(int(element[:2]) + 1) + ":00"
            if len(element) == 5:
                element = element
            else:
                element = "0" + element[0] + ":00"
        return element

    def replace_dot(element):
        element = element.replace('.', ':')
        return element

    def add_seconds_part(element):
        element = element + ":00"
        return element

    data_EB['NEW_PST'] = data_EB['REVISED_PST'].apply(convert_time)
    data_EB['NEW_PET'] = data_EB['REVISED_PET'].apply(convert_time)
    data_EB['NEW_PST'] = data_EB['NEW_PST'].apply(replace_dot)
    data_EB['NEW_PET'] = data_EB['NEW_PET'].apply(replace_dot)
    data_EB['NEW_PST'] = data_EB['NEW_PST'].apply(add_seconds_part)
    data_EB['NEW_PET'] = data_EB['NEW_PET'].apply(add_seconds_part)
    data_EB['REVISED_RUN_TIME'] = data_EB['RUN_TIME'].apply(lambda x: x/600)
    data_EB = data_EB.sort_values(['NEW_PST', 'NEW_PET'])
    data_EB = data_EB.reset_index(drop = True)

    for index, row in data_EB.iterrows():
        if row['NEW_PET'] >= '24:00:00':
            data_EB.loc[index, 'NEW_PET'] = '23:59:59'

    def convert_times(time):
        time_format = "%H:%M:%S"
        time_datetime = datetime.strptime(time, time_format)
        return time_datetime.time()

    y = data_EB[['NEW_PST', 'NEW_PET']].drop_duplicates()
    y = y[(y['NEW_PST'] < '24:00:00') & (y['NEW_PET'] < '24:00:00')]
    y['NEW_PST'] = y['NEW_PST'].apply(convert_times)
    y['NEW_PET'] = y['NEW_PET'].apply(convert_times)

    new_pst_eb = []
    new_pet_eb = []

    y = y.reset_index().drop(['index'], axis = 1).sort_values(by = ['NEW_PST', 'NEW_PET'])

    for i in range(len(y) - 1):
        if (y['NEW_PET'][i] != y['NEW_PST'][i+1]) and (y['NEW_PET'][i] != (datetime.combine(datetime.today(), y['NEW_PST'][i+1]) - timedelta(minutes = 1)).time()) and (y['NEW_PET'][i] < y['NEW_PST'][i+1]):
            new_pst_eb.append(y['NEW_PET'][i])
            new_pet_eb.append(y['NEW_PST'][i+1])

    for i in range(len(new_pet_eb)):
        c = pd.concat([y['NEW_PST'], pd.Series(new_pst_eb)])
        d = pd.concat([y['NEW_PET'], pd.Series(new_pet_eb)])

    c = c.sort_values().reset_index(drop = True)
    d = d.sort_values().reset_index(drop = True)
    df_PST_PET_EB = pd.concat([c, d], axis = 1)
    df_PST_PET_EB = df_PST_PET_EB.rename(columns =  {0 : 'PST', 1: 'PET'})

    comparision_test_dataset_EB = pd.DataFrame(columns = ['Trip', 'Place', 'Time'])
    trip_no = 1
    test_start_time = '03:00:00'
    time_format = "%H:%M:%S"
    test_start_time_datetime_eb = datetime.strptime(test_start_time, time_format)

    if((df_PST_PET_EB['PET'].iloc[-1].hour == 23) and (df_PST_PET_EB['PET'].iloc[-1].minute == 59) and (df_PST_PET_EB['PET'].iloc[-1].second == 59)):
        test_end_time_datetime_eb = datetime(1900, 1, 1, 23, 59, 59)

    else:
        test_end_time_datetime_eb = datetime(1900, 1, 1, df_PST_PET_EB['PET'].iloc[-1].hour, df_PST_PET_EB['PET'].iloc[-1].minute - 1, df_PST_PET_EB['PET'].iloc[-1].second + 59)

    def get_timepoint_runtimes_eb(time):
        data_EB['START-END'] = data_EB['START_PLACE_ID'] + ' - ' + data_EB['END_PLACE_ID']
        return data_EB[(time >= data_EB['NEW_PST']) & (time < data_EB['NEW_PET'])][['START-END', 'REVISED_RUN_TIME']].set_index('START-END').to_dict()

    time = test_start_time

    while test_start_time_datetime_eb <= test_end_time_datetime_eb:
        time_point_runtimes_eb = get_timepoint_runtimes_eb(time)
        time_point_runtime_eb = time_point_runtimes_eb['REVISED_RUN_TIME']

        if len(time_point_runtime_eb) == 0:
            start_time_eb = data_EB[(data_EB['NEW_PET'] <= time)]['NEW_PET'].max()
            end_time_eb = data_EB[(data_EB['NEW_PST'] >= time)]['NEW_PST'].min()
            time_format = "%H:%M:%S"
            start_time_datetime_eb = datetime.strptime(start_time_eb, time_format)
            end_time_datetime_eb = datetime.strptime(end_time_eb, time_format)
            start_time_eb = start_time_datetime_eb
            end_time_eb = end_time_datetime_eb

            dummy_start_time_eb = (start_time_eb - timedelta(minutes = 1)).time()
            dummy_end_time_eb = (end_time_eb + timedelta(minutes = 1)).time()

            p_eb = data_EB[(dummy_start_time_eb.strftime("%H:%M:%S") < data_EB['NEW_PET']) & (dummy_start_time_eb.strftime("%H:%M:%S") >= data_EB['NEW_PST'])]
            n_eb = data_EB[(dummy_end_time_eb.strftime("%H:%M:%S") <= data_EB['NEW_PET']) & (dummy_end_time_eb.strftime("%H:%M:%S") > data_EB['NEW_PST'])]

            timepoint_runtime_dict_previous_eb = {}
            timepoint_runtime_dict_next_eb = {}
            time_point_runtimes_p_ordered_eb = {}
            time_point_runtimes_n_ordered_eb = {}
            time_point_runtimes_p_template_eb = {'IrvCum - IrvHar': 0, 'IrvHar - IrvCen': 0, 'IrvCen - IrvCic': 0,  'IrvCic - IrvBlu': 0, 'IrvBlu - IrvKim': 0, 'IrvKim - IrvWes': 0, 'IrvWes - IrvAsh': 0, 'IrvAsh - SheLs': 0, 'SheLs - IrvBwy': 0}
            time_point_runtimes_n_template_eb = {'IrvCum - IrvHar': 0, 'IrvHar - IrvCen': 0, 'IrvCen - IrvCic': 0,  'IrvCic - IrvBlu': 0, 'IrvBlu - IrvKim': 0, 'IrvKim - IrvWes': 0, 'IrvWes - IrvAsh': 0, 'IrvAsh - SheLs': 0, 'SheLs - IrvBwy': 0}

            for index, row in p_eb.iterrows():
                timepoint_set = row['START-END']
                runtime = row['REVISED_RUN_TIME']
                timepoint_runtime_dict_previous_eb[timepoint_set] = runtime

            for i in time_point_runtimes_p_template_eb:
                time_point_runtimes_p_ordered_eb[i] = timepoint_runtime_dict_previous_eb[i]

            for index, row in n_eb.iterrows():
                timepoint_set = row['START-END']
                runtime = row['REVISED_RUN_TIME']
                timepoint_runtime_dict_next_eb[timepoint_set] = runtime

            for i in time_point_runtimes_n_template_eb:
                time_point_runtimes_n_ordered_eb[i] = timepoint_runtime_dict_next_eb[i]

            time_format = "%H:%M:%S"
            time_for_getting_period_interval_eb = datetime.strptime(time, time_format).time()
            PST_and_PET_for_time_eb = df_PST_PET_EB[(df_PST_PET_EB['PST'] <= time_for_getting_period_interval_eb) & (df_PST_PET_EB['PET'] > time_for_getting_period_interval_eb)]

            PET_for_time_eb = PST_and_PET_for_time_eb['PET'].iloc[0]

            date_object = date(1900, 1, 1)
            PET_for_time_eb = datetime.combine(date_object, PET_for_time_eb)
            PET_for_time_eb = (PET_for_time_eb - timedelta(minutes = 1)).time()

            pet_eb = PET_for_time_eb  
            pst_eb = PST_and_PET_for_time_eb['PST'].iloc[0] 

            pet_datetime_eb = datetime.combine(datetime.today(), pet_eb)
            pst_datetime_eb = datetime.combine(datetime.today(), pst_eb)
            time_datetime_for_getting_period_interval_eb = datetime.combine(datetime.today(), time_for_getting_period_interval_eb)
            time_difference_eb = pet_datetime_eb - pst_datetime_eb

            def y3(x1, x2, x3, y1, y2):
                return (y1 * (x2 - x3) + y2 * (x3 - x1)) / (x2 - x1)

            y_eb = []
            x1 = 0
            x2 = time_difference_eb.seconds + 60
            x3 = (time_datetime_for_getting_period_interval_eb - pst_datetime_eb).seconds

            for i, j in zip(time_point_runtimes_p_ordered_eb, time_point_runtimes_n_ordered_eb):
                y1 = time_point_runtimes_p_ordered_eb[i]
                y2 = time_point_runtimes_n_ordered_eb[j]
                y3_value = y3(x1, x2, x3, y1, y2)
                y_eb.append(y3_value)

            y_eb_new = []

            for i in y_eb:
                integer_part = math.floor(i)
                fractional_part = i - integer_part
                if ((fractional_part >= 0.2500) and (fractional_part <= 0.7499)):
                    fractional_part = 0.5
                    i = integer_part + fractional_part
                elif ((fractional_part >= 0.7500) and (fractional_part <= 0.9999)):
                    integer_part = integer_part + 1
                    fractional_part = 0.0
                    i = integer_part + fractional_part
                elif ((fractional_part >= 0.0000) and (fractional_part <= 0.2499)):
                    fractional_part = 0.0
                    i = integer_part + fractional_part
                else:
                    i = i
                y_eb_new.append(i)

            time_point_runtimes_eb = {'IrvCum - IrvHar': 0, 'IrvHar - IrvCen': 0, 'IrvCen - IrvCic': 0,  'IrvCic - IrvBlu': 0, 'IrvBlu - IrvKim': 0, 'IrvKim - IrvWes': 0, 'IrvWes - IrvAsh': 0, 'IrvAsh - SheLs': 0, 'SheLs - IrvBwy': 0}

            for i, j in zip(time_point_runtimes_eb, range(len(y_eb_new))):
                time_point_runtimes_eb[i] = y_eb_new[j]

            time_format = "%H:%M:%S"
            starting_time_for_cumulative_times_eb = datetime.strptime(time, time_format)

            time_point_order_eb = {'IrvHar': 0, 'IrvCen': 0, 'IrvCic': 0, 'IrvBlu': 0, 'IrvKim': 0, 'IrvWes': 0, 'IrvAsh': 0, 'SheLs': 0, 'IrvBwy' : 0}
            cumulative_times_eb = {'IrvCum': starting_time_for_cumulative_times_eb, 'IrvHar': 0, 'IrvCen': 0, 'IrvCic': 0, 'IrvBlu': 0, 'IrvKim': 0, 'IrvWes': 0, 'IrvAsh': 0, 'SheLs': 0, 'IrvBwy' : 0}

            for i, j in zip(time_point_runtimes_eb, time_point_order_eb):
                time_point_order_eb[j] = time_point_runtimes_eb[i]

            time_point_order_list_eb = list(time_point_order_eb.values())     
            cumulative_times_list_eb = list(cumulative_times_eb.values()) 

            for i, j in zip(range(len(time_point_order_list_eb)), range(len(cumulative_times_list_eb))):
                cumulative_times_list_eb[j+1] = cumulative_times_list_eb[j] + timedelta(minutes = time_point_order_list_eb[i]) 

            for i in range(len(cumulative_times_list_eb)):
                cumulative_times_list_eb[i] = cumulative_times_list_eb[i].time()

            for key, i in zip(cumulative_times_eb, range(len(cumulative_times_list_eb))):
                cumulative_times_eb[key] = cumulative_times_list_eb[i]

        elif len(time_point_runtime_eb) != 0:
            total_run_time_eb = 0

            for i in time_point_runtime_eb:
                total_run_time_eb = total_run_time_eb + time_point_runtime_eb[i]

            time_point_runtimes_eb = {'IrvCum - IrvHar': 0, 'IrvHar - IrvCen': 0, 'IrvCen - IrvCic': 0,  'IrvCic - IrvBlu': 0, 'IrvBlu - IrvKim': 0, 'IrvKim - IrvWes': 0, 'IrvWes - IrvAsh': 0, 'IrvAsh - SheLs': 0, 'SheLs - IrvBwy': 0}

            for i in time_point_runtime_eb:
                time_point_runtimes_eb[i] = time_point_runtime_eb[i]

            time_format = "%H:%M:%S"
            time = datetime.strptime(time, time_format)

            time_point_order_eb = {'IrvHar': 0, 'IrvCen': 0, 'IrvCic': 0, 'IrvBlu': 0, 'IrvKim': 0, 'IrvWes': 0, 'IrvAsh': 0, 'SheLs': 0, 'IrvBwy' : 0}
            cumulative_times_eb = {'IrvCum': time, 'IrvHar': 0, 'IrvCen': 0, 'IrvCic': 0, 'IrvBlu': 0, 'IrvKim': 0, 'IrvWes': 0, 'IrvAsh': 0, 'SheLs': 0, 'IrvBwy' : 0}

            for i, j in zip(time_point_runtimes_eb, time_point_order_eb):
                time_point_order_eb[j] = time_point_runtimes_eb[i]

            time_point_order_list_eb = list(time_point_order_eb.values())     
            cumulative_times_list_eb = list(cumulative_times_eb.values()) 

            for i, j in zip(range(len(time_point_order_list_eb)), range(len(cumulative_times_list_eb))):
                cumulative_times_list_eb[j+1] = cumulative_times_list_eb[j] + timedelta(minutes = time_point_order_list_eb[i]) 

            for i in range(len(cumulative_times_list_eb)):
                cumulative_times_list_eb[i] = cumulative_times_list_eb[i].time()

            for key, i in zip(cumulative_times_eb, range(len(cumulative_times_list_eb))):
                cumulative_times_eb[key] = cumulative_times_list_eb[i]

        else:
            pass

        for i in cumulative_times_eb:
            comparision_test_dataset_EB.loc[len(comparision_test_dataset_EB.index)] = [trip_no, i, cumulative_times_eb[i]]

        trip_no = trip_no + 1
        test_start_time_datetime_eb = test_start_time_datetime_eb + timedelta(seconds = 30)
        time = test_start_time_datetime_eb.strftime("%H:%M:%S")
    
    return comparision_test_dataset_EB